# 3. Classification

## Load utilities and programming set up

Let's start by loading the useful libraries, specifying the input data file, and preparing a utility function that will be used to save any figures later on.

In [20]:
#%pip install import-ipynb

In [21]:
import import_ipynb
from _1_DataCleaning import save_fig

In [22]:
### Read the dataset form the previous step
import pandas as pd
credit_prepared, credit_labels_prepared = pd.read_pickle("dataset_train_step2.pickle")
credit_prepared

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
1602,Attrited Customer,54,F,3,College,Single,Unknown,Blue,49,6,2,3,13184.0,0,1.166,2047,33,0.179,0.000
7791,Existing Customer,51,M,0,High School,Single,$80K - $120K,Blue,45,3,2,3,10648.0,2517,0.708,4930,89,0.978,0.236
7177,Existing Customer,45,F,4,High School,Single,$40K - $60K,Blue,29,3,2,1,4595.0,1781,0.708,3746,68,0.545,0.388
97,Existing Customer,53,M,3,Doctorate,Married,$40K - $60K,Blue,35,5,3,2,3789.0,1706,1.047,1609,42,0.680,0.450
4820,Existing Customer,48,F,2,Graduate,Married,Unknown,Blue,40,3,2,4,5104.0,736,0.679,3943,78,0.696,0.144


We will plot the column distributions using the `scatter_matrix(.)` function.
The resulting array of plots shows:
- each column distribution in the diagonal subplots
- correlations between respecive pairs of columns in the non-diagonal subplots

# Choose and train a ML model

In [26]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(credit_prepared, credit_labels_prepared)

LinearRegression()

In [28]:
# Let's try it out
some_data = df.iloc[:5]
some_labels = credit_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

Predictions: [0.42865919 1.23814271 0.89783302 0.87524327 0.8181202 ]
Labels: ['Attrited Customer', 'Existing Customer', 'Existing Customer', 'Existing Customer', 'Existing Customer']


In [29]:
from sklearn.metrics import mean_squared_error
credit_predictions = lin_reg.predict(credit_prepared)
lin_mse = mean_squared_error(credit_labels_prepared, credit_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.29075752512231773

In [31]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(credit_prepared, credit_labels_prepared)

credit_predictons = tree_reg.predict(credit_prepared)
tree_mse = mean_squared_error(credit_labels_prepared, credit_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.29075752512231773

In [33]:
# the above looks fishy -> same number?
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, credit_prepared, credit_labels_prepared, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(tree_rmse_scores)

Scores: [0.25563923 0.21659543 0.21942686 0.27216553 0.248452   0.22771002
 0.24088315 0.25337232 0.25819889 0.22771002]
Mean: 0.24201534405531114
Standard deviation: 0.017589600745373773


In [34]:
lin_scores = cross_val_score(lin_reg, credit_prepared, credit_labels_prepared,
                            scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [0.2904641  0.29092639 0.27747691 0.30298606 0.30020822 0.2985719
 0.29140202 0.2784484  0.29261424 0.29044852]
Mean: 0.2913546759788459
Standard deviation: 0.007926622475223289


The first column shows a unique identifier for each client;  it will prove useful in randomly splitting into train- and test-set. Although explicitly there are no null values, some of them are marked as e.g. 'unknown' and should be considered as null values.
The data provider suggested to remove the last two columns (they might be some work-in-progress development...).


# Create a Test Test

For simplicity and because the dataset is reasonably large (10k customers), a random split should suffice and bring a roughly homogenous test sample.

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

Project TODOs:

    + plot features for 'signal' and 'background'
    - decide when to split into train/test set and how (k-fold)
    - consider 'balanced' training 'signal' and 'background'
    - train a bunch of models
    - shortlist the best ones
    - fine tune them
    - deploy the result